Repovizz Authentication
=======================

To make requests to the API, a Client **will have to be authorized by both the user and repovizz**. To do that, the Client has first to request repovizz, which will authenticate and ask the user, to then generate a token that can be used by the client to access the API on behalf of the user.

Some clarifying Oauth2 concepts:
- _User_: An account registered in repoVizz. Users have usernames and passwords.
- _Client_: A piece of software that access repoVizz on behalf of the User. Clients have ID and Secrets. Our example scripts are clients.
- _Provider_: RepoVizz. It holds data and functionality of the User.

So to run these scripts, you will need a repoVizz account. [You can create one for free here](http://repovizz2.upf.edu/user/register). One you have don that, you can register a Client. To do so, go to [new client form](http://repovizz2.upf.edu/new_client) and fill the details:
- _Name_: The client's name
- _Description_: The client's purpose
- _Privacy_: Wether this client is for _public_ or _private_ use.
- _Scopes_: Depending of the actions this client will perfor (reading datapacks, changing account details) you have to request a series of _scopes_. RepoVizz implements the following scopes (repoVizz will be adding more scopes):
     - _basic_: This scope gives the Client full powers.
- _Redirect URLs_: List of urls for redirection after authentication, one per line. To run the examples you only need to add `http://localhost:50009`. See [Redirects](#Redirects) for more info.


After registering your client, you can fetch its _ID_ and _Secret_. Do it in the [clients list page](http://repovizz2.upf.edu/clients). Please keep the client secret, you know, _secret_. With those you can start using a script (Client) to perform actions through repovizz API.

Repovizz Authentication Flow, briefly
-------------------------------------

Repovizz, as an oauth2 provider, defines two endpoints that will be used:

- Authorization endpoint, used to ask the user for permission
- Token endpoint, used to request a token

The Client will craft a url using the authorization point, its ID, the requested scope and a _redirection url_. The user must open this url in a web browser, authenticate herself, and confirm the permissions requested by the client. After doing so, she will be redirected to the _redirection url_ with some of the parameters encoded in. The client can use this redirection to acquire a temporary token that can be used later.

Using the token endpoint, the client will create a url encoding the temporary token to get the access token that will be attached in all the repoVizz API calls. No user intervention is required then.

This way the client will never ask the User for the username or password.

Redirects
---------

When registering a client, we have to provide a list of redirects. Those are the ones that can be requested by the client in the authorization phase. Usually Web applications will use redirects in its own domain, so they can process the temporary token and acquire the access token automatically. Desktop applications can set up a temporary web server to do so (redirecting to `loclahost`) or rely on an external service. RepoVizz also provides a generic redirect that can be used by clients that don't want to implement it: `/oauth/show_token`; this will show the url to the user so she can copy and paste it to the client manually.
 

Authenticating a client using `repovizz` Python library
-------------------------------------------------------

This repository has the `repovizz` client python library that eases the process of the authentication:

In [ ]:
from repovizz import RepoVizzClient

# Creating client, repovizz_url is optional, used for testing purposes
repovizz_client = RepoVizzClient(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    repovizz_url="http://repovizz2.upf.edu")

# This creates the authorization url
authorization_url = repovizz_client.start_auth()

# Start internal webserver
# If async is False or absent this call will block

repovizz_client.start_server(async=True)

# open this authorization url in the web browser
import webbrowser
webbrowser.open(authorization_url)

# This will block until user confirmation and will finish the authentication
repovizz_client.finish_auth()

# We can now start using the API
repovizz.get(...)
repovizz.post(...)